In [1]:
import pandas as pd

In [56]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]
df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(2)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,1,501,1,2019-02-01 00:00:07,2019-02-01,1,2,2019,00:00:07,0,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
1,2,502,2,2019-02-01 00:00:09,2019-02-01,1,2,2019,00:00:09,0,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN


In [193]:
# Nome para colunas desconhecidas
colnames = ['x', 'y', 'z']
# Usando separador '.' apenas para conseguir importar e depois tentar tratar
df_ocr = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='.', names = colnames)

def joinStrings(x):
    result = ''
    for string in x.split():
        if string[0].isdigit():
            result = '|'.join([result,string])
        else:
            result = ' '.join([result,string])
    return result
            
# Separando as colunas manualmente
df_ocr['placa'] = df_ocr.x.apply(lambda x: x.split()[0])
df_ocr['a'] = df_ocr.x.apply(lambda x: x.split()[1])
df_ocr['b'] = df_ocr.x.apply(lambda x: x.split()[2])
df_ocr['c'] = df_ocr.x.apply(lambda x: joinStrings(x.split('St')[0]).split('|')[3])
df_ocr['bairro'] = df_ocr.x.apply(lambda x: x.split('St')[1][:-4])
df_ocr['temp'] = df_ocr.x.apply(lambda x: x.split('St')[1][-4:])
df_ocr['e'] = df_ocr['temp']+df_ocr['y']+df_ocr['z'].astype(str)
df_ocr.drop(['x','temp', 'y', 'z'], axis = 1, inplace=True)

# print(df_ocr.x[9].split('St'))
# print(df_ocr.c[0].split())
df_ocr.head(3)



,placa,a,b,c,d,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455,04101911120252015R Conde Baependi Px141,R Catete,B-2293380-4318294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410,04101211220252015Est Bandeirantes Px7967,Tanque,D-2297019-4341377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436,04102411110252015Av Paulo Frontin Px763,Cidade Nova,A-2293149-4320888
